In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from random import randint
from numpy import array
from numpy import argmax
from random import randint
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


from math import sin
from math import pi
from math import exp
from random import randint
from random import uniform

from numpy import array
from math import ceil
from math import log10
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

from random import random
from numpy import cumsum
from numpy import array_equal
from keras.layers import Bidirectional

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [ ]:
log_df = pd.read_parquet("logs.parquet")

In [ ]:
# Feature transformations

# Convert resp_time and bytes_setn into float

log_df.resp_time = log_df.resp_time.astype("float")
log_df.bytes_sent = log_df.bytes_sent.astype("float")

# Convert datetime string into Pandas Datetime

log_df.datetime = pd.to_datetime(log_df.datetime,
                                    format="%Y-%m-%d %H:%M:%S")

# Create categorical variables

log_df = pd.get_dummies(log_df, columns = ['resp_code'])
log_df = pd.get_dummies(log_df, columns = ['url'])

In [ ]:
# Create url request only dataframe

log_df.columns.tolist()

url_features = [col for col in log_df.columns if "url" in col]

req_df = log_df[url_features]

req_df.shape

# Create subset (every 10th request) and trim

small_req_df = req_df.iloc[::2, :]

small_req_df.shape

In [ ]:
# Create sequence prediction outcome variable -> next 250 requests

for url in url_features:
     small_req_df["CS_" + url] = small_req_df[url].rolling(250).sum()

cumsum_cols = [col for col in small_req_df.columns if "CS_" in col]
        
# Delete NaN rows

small_req_df.dropna(inplace=True)

# Shift output variable 250 places down

small_req_df[cumsum_cols] = small_req_df[cumsum_cols].shift(-250)

# Delete NaN rows again

small_req_df.dropna(inplace=True)

In [ ]:
small_req_df.shape

In [ ]:
# trim to size

small_req_df = small_req_df.iloc[0:2241000]

In [ ]:
# Split into X and y

y = array(small_req_df[cumsum_cols]).reshape(2241000,129)


X = small_req_df[[col for col in small_req_df.columns if "CS_" not in col]].to_numpy()
X = X.reshape(2241,1000,129)

# Get the thousandth request vector for y

sub_y = y[::1000]

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1000, 129)))
model.add(LSTM(50))
model.add(Dense(129))
model.compile(loss='cosine_proximity', optimizer='adam')
model.summary()

# fit model
history = model.fit(X, sub_y, batch_size=32, epochs=1)




'''
# evaluate model
X, y = generate_examples(length, 1000, output)
loss = model.evaluate(X, y, verbose=0)
print('MAE: %f' % loss)

# prediction on new data
X, y = generate_examples(length, 1, output)
yhat = model.predict(X, verbose=0)
pyplot.plot(y[0], label='y')
pyplot.plot(yhat[0], label='yhat')
pyplot.legend()
pyplot.show()
'''

In [ ]:
test = X[2100].reshape(1,1000,129)

In [ ]:
model.predict(test)

In [ ]:
y[2100]